## Percentage on Fluids and Vasopressor treatments

### Convenience Functions

In [1]:
import os
import numpy as np
import pandas as pd
from tqdm import tqdm

CHUNK_SIZE = 10000

In [2]:
def read_csv(path, usecols, chunksize=CHUNK_SIZE):
    """ Reads file limited to columns in `usecols` from path in chunks of size `chunksize` """
    for i, chunk in enumerate(pd.read_csv(path, usecols=usecols, encoding='latin1', engine='c', chunksize=chunksize)):
        yield i, chunk.reset_index(drop=True) # resets index so that indices range from 0 to chunksize - 1

In [3]:
def pbar(iterator, col='admissionid', total_admissions=23106):
    # Keep track of admissions already seen
    processed_admissions = set()
    
    with tqdm(total=total_admissions) as progress_bar:
        for i, chunk in iterator:
            
            # count number of new admissions not yet seen
            # we do it only if last admission is new (saves time checking)
            if chunk[col].values[-1] not in processed_admissions:
                new_admissions = set(chunk[col]) - processed_admissions
                processed_admissions.update(new_admissions)
            
                # update progress bar
                progress_bar.update(len(new_admissions))
                
            yield i, chunk

## AmsterdamUMCdb

In [7]:
admissions_with_iv = set()
admissions_with_vp = set()
admissions_all = set()

for i, chunk in pbar(read_csv(r"D:/AmsterdamUMCdb-v1.0.2/drugitems.csv", usecols=['admissionid', 'ordercategoryid', 'item', 'itemid'])):
    admissions_with_iv.update(chunk.admissionid[(chunk.ordercategoryid == 17) | (chunk.ordercategoryid == 55)].unique()) # colloids/crystalloids
    admissions_with_vp.update(chunk.admissionid[chunk.itemid.isin([7229, 7179, 6818, 19929])].unique()) # norepinephrine/dopamine/epinefrine/phenylphrine -> vasopressin not administered!
    admissions_all.update(chunk.admissionid)

print('IV: %.2f%%' % (100 * len(admissions_with_iv) / len(admissions_all)))
print('VP: %.2f%%' % (100 * len(admissions_with_vp) / len(admissions_all)))

100%|█████████████████████████████████████████████████████████████████████████▋| 22999/23106 [00:16<00:00, 1425.84it/s]

IV: 99.60%
VP: 58.81%


## MIMIC-III

In [8]:
VP_MIMIC = [
    30120,221906,30047, # norepinephrine
    221662,30043,30307, # dopamine
    30119,221289,       # epinefrine
    30128,221749,30127, # phenylphrine
    30051,222315        # vasopressin
]

IV_CATEGORIES_MIMIC = [
    '01-Drips',
    '02-Fluids (Crystalloids)',
    '04-Fluids (Colloids)',
    '03-IV Fluid Bolus',
    '10-Prophylaxis (IV)',
    '07-Blood Products',
]

IV_ITEMIDS_MIMIC = [
    225158,225943,226089,225168,225828,225823,220862,220970,220864,225159,220995,225170,
    225825,227533,225161,227531,225171,225827,225941,225823,225825,225941,225825,228341,
    225827,30018,30021,30015,30296,30020,30066,30001,30030,30060,30005,30321,30006,30061,
    30009,30179,30190,30143,30160,30008,30168,30186,30211,30353,30159,30007,30185,30063,
    30094,30352,30014,30011,30210,46493,45399,46516,40850,30176,30161,30381,30315,42742,
    30180,46087,41491,30004,42698,42244,220949,225944,227523   # a lot of IDs....
]

In [9]:
admissions_with_iv = set()
admissions_with_vp = set()
admissions_all = set()

mimic_data = read_csv(r"D:/mimic-iii-clinical-database-1.4/INPUTEVENTS_MV.csv", usecols=['ICUSTAY_ID', 'ITEMID', 'AMOUNTUOM', 'ORDERCATEGORYNAME'])

for i, chunk in pbar(mimic_data, col='ICUSTAY_ID', total_admissions=21247):
    admissions_with_iv.update(chunk.ICUSTAY_ID[chunk.ORDERCATEGORYNAME.isin(IV_CATEGORIES_MIMIC) & chunk.ITEMID.isin(IV_ITEMIDS_MIMIC) & (chunk.AMOUNTUOM == 'ml')].unique())
    admissions_with_vp.update(chunk.ICUSTAY_ID[chunk.ITEMID.isin(VP_MIMIC)].unique())
    admissions_all.update(chunk.ICUSTAY_ID)
    
print('IV: %.2f%%' % (100 * len(admissions_with_iv) / len(admissions_all)))
print('VP: %.2f%%' % (100 * len(admissions_with_vp) / len(admissions_all)))

100%|██████████████████████████████████████████████████████████████████████████| 21247/21247 [00:13<00:00, 1529.07it/s]

IV: 92.30%
VP: 28.72%
